Lets simulate a CRN. We can run a loop for a set number of iterations where each iteration the sensor network can allocate channels to a primary user and secondary users can sense(query) and use unused channels, how they use the channel will affect their trustValue as assessed by other users. Every some number of iterations all the users will use trimessage communication to synchronize their trust values. Each iteration a fraction of secondary users can also try to be malicious by giving other users lower trust scores, giving themself a high trust score, or lying about whether the primary user is in the channel. Since we don't have a data fusion center, other users will have to "catch" these lies and adjust trust scores accordingly.

In [32]:
import math
import random
import matplotlib.pyplot as plt
import numpy as np

In [23]:
num_iterations = 5000

In [37]:
class SensorNetwork:
    """
    Can only read values from nodes, not write them (decentralized network)
    - Know if primary user is active
    - Know what Secondary Users exist, and what they believe/do
    """
    users = [] #devices in the network
    numChannels = 1 #available channels
    data = [] #row 0 = PU, row 1-n = SU; z0 = channel use, z1=belief
    clock = 0

    def __init__(self, userlist, primaryuser):
        self.users = userlist
        self.primaryuser = primaryuser
        self.data = np.zeros(shape = (len(users)+1, num_iterations, 2))

    def update_users(self):
        """
        Advance each Secondary User's clock one step
        :return:
        """
        signal = random.randint(0, 1)
        i = 0
        #data[i, clock, 0] = signal
        
        for user in self.users:
            i += 1
            user.update(self.primary_user_strength(user, signal), self.clock, 0)
            #data[i, clock, 0] = user.get_action()
            #data[i, clock, 1] = user.get_belief()
            
        self.clock += 1
        return signal
        
    def primary_user_strength(self, user, signal):
        distance = math.sqrt((user.x - self.primaryuser.x)**2 + (user.y - self.primaryuser.y)**2)
        noise = (random.random()-0.5)/2
        return signal + noise * distance**2

In [89]:
class User:
    id_value = 0  # this user's id for distinguishability
    channelAllocated = False
    timeSinceAllocation = 0  # elapsed time since this device used a channel
    x = 0 # x position
    y = 0 # y position

    def __init__(self, id_val, x, y):
        self.id_value = id_val
        self.x = x
        self.y = y
    
class SecondaryUser(User):
    """
    A "good faith" SU, no malicious behavior
    """
    trust_value = 100 # The user's evaluation of its own trustworthiness, scaled 0 to 200
    trust_values = []  # The perceived trust values of other users
    user_list = []
    channel_allocated = False
    primary_user_value = False # belief on whether or not primary user is in the channel
    
    def __init__(self, id_val, x, y,):
        super().__init__(id_val, x, y)

    def set_users(self, neighbors):
        self.user_list = neighbors
        self.trust_values = np.full((len(neighbors),1), 100)
        
    # Inspired by the following StackOverflow thread
    # https://stackoverflow.com/questions/10324015/fitness-proportionate-selection-roulette-wheel-selection-in-python
    def roulette(self):
        max_CTV = sum([i.trust_value for i in self.user_list])
        selection_probs = [i.trust_value/max_CTV for i in self.user_list]
        return self.user_list[random.choice(len(self.user_list), p=selection_probs)]
    
    def synchronize_trust(self):
        """
        Communicate/receive trust values from other users.
        Ignore broadcast procedures, they can get values from other users at any time.
        :return:
        """
        beliefs = []

        for i in range(len(self.user_list)):
            # Iterate through the list of all users, get their belief 
            # of if the PU is present 
            # get_belief returns a bool, which we convert to an int
            beliefs.append(int(self.user_list[i].get_belief()))
        
        # if this is true then 2/3 of nodes report
        # the PU is present
        if sum(beliefs) >= len(self.user_list) * 2/3: 
            MU_index = beliefs[0]
            self.user_list[MU_index].trust_value = max(self.user_list[MU_index].trust_value - 10, 0) # bound CTVs between 0 and 200
    
        # if this is true all users are reporting that 
        # the PU is NOT present except for 1 - the MU 
        elif sum(beliefs) < len(self.user_list) * 1/3:
            MU_index = beliefs[1]
            self.user_list[MU_index].trust_value = max(self.user_list[MU_index].trust_value - 15, 0) # bound CTVs between 0 and 200

        else:
            mean = self.trust_value
            # as there has been no detection of an SSDF attack by either devices 
            # the nodes are rewarded by having their trust values incremented by 5
            for i in range(len(self.user_list)):
                mean += self.user_list[i].get_trust_value()
            
            mean = (1/len(self.user_list))*mean
            
            for u in self.user_list:    
                u.set_trust_value(min(mean + 5, 200))# bound CTVs between 0 and 200
            self.trust_value = min(mean +  5, 200) # bound CTVs between 0 and 200
        
    def broadcast(self):
        """
        broadcast values to all other users
        :return:            
        """
        # For all users in the network, make them recieve 
        # the trust values of this particular instance
        for i in range(len(self.user_list)):
                self.user_list[i].receive_broadcast(self.trust_values, self.trust_value)
    
    def receive_broadcast(self, trustValues, trust_value_other_device):
        """
        Receive a broadcast of trust values, and adjust own accordingly.
        :return:
        """
        # compare the nodes trust values and use that as a weight to update the trust 
        # values of the device
        sum_of_trust_values = self.trust_value + trust_value_other_device
        #print(sum_of_trust_values)
        ratio = trust_value_other_device/sum_of_trust_values
        self.trust_values = self.trust_values + (ratio)*trustValues
    
    def update(self, primary_user_value, clock, channel_value):
        """
        Decide what actions to take, synchronize trust? Broadcast values? Use channel?
        :return:
        """
        self.clock = clock
        self.primary_user_value = primary_user_value
        x = random.random()
        
        if x < 0.3334: #take channel process
            if primary_user_value == 0 and channel_value == 0:
                # if the the id of the device matches the id of 
                # the device chosen from our roulette wheel function 
                # allocate it the channel
                if self.id_val == self.roulette().id_val: # highest in network
                    self.channelAllocated = True
        elif x < 0.6667: # tri-message trust sync
            self.synchronize_trust()
        else: # broadcast
            self.broadcast()
                    
            
    
    def get_action(self):
            """:
            return: boolean true if channel is allocated to this SU 
            """
            return self.channel_allocated
    
    def get_belief(self):
            """
            :return: boolean true if channel is believed to be allocated to PU
            """
            return self.primary_user_value
    
    def get_trust_value(self):
            """
            :return: int the trust value
            """
            return self.trust_value
    def set_trust_value(self, val):
            self.trust_value = val



In [28]:
class MaliciousUser(SecondaryUser):
    """
    Users with potential to be malicious, some strategy:
    -Selfish SSDF: Falsely report that PU is present, in order to gain exclusive access to the desired
    spectrum.
    - Interference SSDF: Falsely report that PU is not present, in order to cause interference to PU.
    - Confusing SSDF: Randomly reports a true or false value for primary user energy, preventing a
    consensus.
    """
    
    def SelfishSSDF(self):
        if not self.get_belief():
            self.primary_user_value = True 

    def InterferenceSSDF(self):
        if self.get_belief():
            self.primary_user_value = False
    
    def ConfusingSSDF(self):
        z = random.random()
        if z < 0.5:
            self.primary_user_value = True 
        else:
            self.primary_user_value = False

In [57]:
def generate_point(mean_x, mean_y, deviation_x, deviation_y):
    return random.gauss(mean_x, deviation_x), random.gauss(mean_y, deviation_y)


def initializeUsers():
    users = []
    # Add 1 primary User
    # Add Secondary Users to network
    cluster_mean_x = 1
    cluster_mean_y = 1
    cluster_deviation_x = 1
    cluster_deviation_y = 1
    point_deviation_x = 0.2
    point_deviation_y = 0.2

    number_of_clusters = 5
    points_per_cluster = 50

    cluster_centers = [generate_point(cluster_mean_x,
                                      cluster_mean_y,
                                      cluster_deviation_x,
                                      cluster_deviation_y)
                       for _ in range(number_of_clusters)]

    points = [generate_point(center_x,
                             center_y,
                             point_deviation_x,
                             point_deviation_y)
              for center_x, center_y in cluster_centers
              for _ in range(points_per_cluster)]

    for idx, p in enumerate(points):
        users.append(SecondaryUser(idx, p[0], p[1]))
        
    for user in users:
        user.set_users(users)
        
    print(len(users))
    return users


def plot_users(users, primary_user):
    plt.scatter([user.x for user in users], [user.y for user in users])
    plt.scatter(primary_user.x, primary_user.y, c="red")
    plt.show()


In [90]:
if __name__ == '__main__':
    users = initializeUsers()
    net = SensorNetwork(users, SecondaryUser(-1, 1, 1))
    vals = []
    for i in range(num_iterations):
        net.update_users()
    plot_users(users, SecondaryUser(-1, 1, 1))

250


ZeroDivisionError: division by zero